In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ast
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, ReLU, Attention, GlobalAveragePooling1D, Concatenate, Flatten, Softmax, MultiHeadAttention, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

In [2]:
# Read both CSV files
attack_params = pd.read_csv("dataset/attack/attack_parameters.csv")
simulation_params = pd.read_csv("dataset/nonattack/simulation_parameters.csv")

In [3]:
attack_params.head()

,testcase,ue_list,noise_db,tx_power_db,start_time,end_time
0,1,"2,3",26.19,12.87,1.021183,1.688284
1,2,4,19.32,25.52,0.775680,1.229469
2,3,"3,2",19.66,21.28,1.295688,1.985717
3,4,"4,2,1",19.53,15.13,0.597156,1.755974
4,5,"3,5,6,1,7,2",19.63,10.37,0.453392,0.836695


In [4]:
simulation_params.head()

,test_case,n,ue_posn,ue_vel
0,1,3,"-37,36,0,-46,49,0,-30,21,0","-6,2,0,-10,7,0,-7,7,0"
1,2,4,"12,-11,0,-24,-12,0,20,41,0,-4,-6,0","8,-10,0,9,-1,0,0,9,0,-4,-5,0"
2,3,5,"-35,-35,0,34,-9,0,42,30,0,33,18,0,-43,-48,0","-1,9,0,-4,0,0,-7,8,0,8,-2,0,0,8,0"
3,4,6,"-28,4,0,29,17,0,-45,-49,0,10,-42,0,-40,8,0,-31...","-6,1,0,-7,9,0,7,-3,0,-2,6,0,-8,4,0,-5,-9,0"
4,5,7,"29,-22,0,-27,-33,0,29,27,0,10,7,0,7,-1,0,40,0,...","-8,-8,0,-5,5,0,3,2,0,-1,8,0,-4,0,0,-1,4,0,-7,-3,0"


In [5]:
# Create a dictionary to store n for each testcase (assuming testcase is the index)
# Adjust this based on how n is stored in simulation_parameters.csv
testcase_to_n = dict(zip(simulation_params['test_case'], simulation_params['n']))
testcase_to_n

{1: 3,
 2: 4,
 3: 5,
 4: 6,
 5: 7,
 6: 8,
 7: 9,
 8: 10,
 9: 2,
 10: 3,
 11: 4,
 12: 5,
 13: 6,
 14: 7,
 15: 8,
 16: 9,
 17: 10,
 18: 2,
 19: 3,
 20: 4,
 21: 5,
 22: 6,
 23: 7,
 24: 8,
 25: 9,
 26: 10,
 27: 2,
 28: 3,
 29: 4,
 30: 5,
 31: 6,
 32: 7,
 33: 8,
 34: 9,
 35: 10,
 36: 2,
 37: 3,
 38: 4,
 39: 5,
 40: 6,
 41: 7,
 42: 8,
 43: 9,
 44: 10,
 45: 2,
 46: 3,
 47: 4,
 48: 5,
 49: 6,
 50: 7,
 51: 8,
 52: 9,
 53: 10,
 54: 2,
 55: 3,
 56: 4,
 57: 5,
 58: 6,
 59: 7,
 60: 8,
 61: 9,
 62: 10,
 63: 2,
 64: 3,
 65: 4,
 66: 5,
 67: 6,
 68: 7,
 69: 8,
 70: 9,
 71: 10,
 72: 2,
 73: 3,
 74: 4,
 75: 5,
 76: 6,
 77: 7,
 78: 8,
 79: 9,
 80: 10,
 81: 2,
 82: 3,
 83: 4,
 84: 5,
 85: 6,
 86: 7,
 87: 8,
 88: 9,
 89: 10,
 90: 2,
 91: 3,
 92: 4,
 93: 5,
 94: 6,
 95: 7,
 96: 8,
 97: 9,
 98: 10,
 99: 2,
 100: 3,
 101: 4,
 102: 5,
 103: 6,
 104: 7,
 105: 8,
 106: 9,
 107: 10,
 108: 2,
 109: 3,
 110: 4,
 111: 5,
 112: 6,
 113: 7,
 114: 8,
 115: 9,
 116: 10,
 117: 2,
 118: 3,
 119: 4,
 120: 5,
 121: 6,
 122: 

In [6]:
# Initialize the result array
result_labels = []

for _, row in attack_params.iterrows():
    testcase = row['testcase']
    ue_list_str = row['ue_list']
    
    # Convert ue_list to a list of integers
    if "," in ue_list_str:
        ue_list = [int(x) for x in ue_list_str.split(",")]
    else:
        ue_list = [int(ue_list_str)]  # Single value becomes a single-item list
    
    # Get total number of UEs for this testcase
    n = testcase_to_n[testcase]
    
    # Initialize all labels as 0
    labels = [0] * n
    
    # Set the specified UEs to 1
    for ue in ue_list:
        # Assuming UE numbers start from 1 (if they start from 0, remove the -1)
        if ue <= n:  # Safety check to avoid index errors
            labels[ue - 1] = 1
    
    result_labels.append(labels)

# Now result_labels contains your array of arrays
print(result_labels)

[[0, 1, 1], [0, 0, 0, 1], [0, 1, 1, 0, 0], [1, 1, 0, 1, 0, 0], [1, 1, 1, 0, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 0], [1, 1, 1], [1, 1, 1, 1], [0, 0, 0, 1, 0], [0, 1, 1, 1, 0, 1], [1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 1, 1, 0, 1, 1, 0, 0, 1], [0, 1], [0, 1, 1], [0, 0, 0, 1], [1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 0, 1], [1, 1, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 1, 1, 0, 0, 0, 1, 0, 0, 1], [0, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 0], [0, 1, 0, 0, 0, 1], [1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 1], [1, 0, 0], [1, 1, 1, 1], [0, 0, 1, 0, 1], [1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 1, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 1, 1, 0, 0, 1, 0], [1, 1], [0, 1, 1], [1, 0, 1, 0], [1, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1], [0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0

In [7]:
print("Number of test cases:", len(result_labels))

Number of test cases: 550


In [8]:
training_labels = result_labels[:90]  # Takes first 90 sub-arrays
flattened_array = np.concatenate(training_labels)
print(flattened_array.shape)

(540,)


In [10]:
flattened_array # these are the labels for training

array([0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [38]:
# Assuming flattened_array is your label array [0,0,1,1,0,...]
labels = np.array(flattened_array)  # Convert to numpy array if not already

# One-hot encode (will create 2 columns: [1,0] for class 0, [0,1] for class 1)
one_hot_labels = to_categorical(labels, num_classes=2)

print("Original label shape:", labels.shape)
print("One-hot encoded shape:", one_hot_labels.shape)

Original label shape: (540,)
One-hot encoded shape: (540, 2)


In [11]:
# see sample of shapes of time series
rssi_exp = pd.read_csv("dataset/attack/TC1/UE1_rssi.csv")
sinr_exp = pd.read_csv("dataset/attack/TC1/UE1_sinr.csv")
print(rssi_exp.shape)
print(sinr_exp.shape)

(63862, 2)
(15946, 2)


In [12]:
def reduce_and_match_shapes(df_rssi, df_sinr=None, group_size=4, truncate=True):
    # --- Step 1: Reduce RSSI (average duplicates + group every `group_size`) ---
    df_rssi_reduced = df_rssi.groupby('Time', as_index=False)['RSSI'].mean()
    
    # Group every `group_size` rows
    n = len(df_rssi_reduced)
    num_groups = n // group_size
    
    # Reshape and aggregate
    time_median = np.median(
        df_rssi_reduced['Time'].values[:num_groups * group_size].reshape(-1, group_size),
        axis=1
    )
    rssi_mean = np.mean(
        df_rssi_reduced['RSSI'].values[:num_groups * group_size].reshape(-1, group_size),
        axis=1
    )
    
    df_rssi_final = pd.DataFrame({'Time': time_median, 'RSSI': rssi_mean})
    print(f"RSSI reduced shape: {df_rssi_final.shape} (from original {df_rssi.shape})")

    # --- Step 2: Match shapes if SINR is provided ---
    if df_sinr is not None and truncate:
        len_rssi = len(df_rssi_final)
        len_sinr = len(df_sinr)
        
        if len_rssi > len_sinr:
            df_rssi_final = df_rssi_final.iloc[:len_sinr]  # Truncate RSSI
            print(f"Truncated RSSI to match SINR: {df_rssi_final.shape}")
        elif len_sinr > len_rssi:
            df_sinr = df_sinr.iloc[:len_rssi]  # Truncate SINR
            print(f"Truncated SINR to match RSSI: {df_sinr.shape}")
        else:
            print("RSSI and SINR already have the same length.")
        
        return df_rssi_final, df_sinr
    else:
        return df_rssi_final

In [13]:
rssi_reduced, sinr_reduced = reduce_and_match_shapes(rssi_exp, sinr_exp, group_size=4)

RSSI reduced shape: (13965, 2) (from original (63862, 2))
Truncated SINR to match RSSI: (13965, 2)


In [ ]:
import os

# Initialize lists to store lengths
rssi_lengths = []
sinr_lengths = []

# Path to your dataset
base_path = "dataset/training/"

# Process RSSI files
for i in range(1, 541):
    filename = f"rssi_{i}.csv"
    filepath = os.path.join(base_path, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        rssi_lengths.append(len(df)/4)
    else:
        print(f"Warning: {filename} not found")
        rssi_lengths.append(0)  # or np.nan if you prefer

# Process SINR files
for i in range(1, 541):
    filename = f"sinr_{i}.csv"
    filepath = os.path.join(base_path, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        sinr_lengths.append(len(df))
    else:
        print(f"Warning: {filename} not found")
        sinr_lengths.append(0)  # or np.nan if you prefer

# Convert to numpy arrays
rssi_lengths = np.array(rssi_lengths)
sinr_lengths = np.array(sinr_lengths)

# Calculate statistics
rssi_median = np.median(rssi_lengths)
rssi_mean = np.mean(rssi_lengths)

sinr_median = np.median(sinr_lengths)
sinr_mean = np.mean(sinr_lengths)

print("RSSI lengths statistics:")
print(f"Median: {rssi_median}")
print(f"Mean: {rssi_mean}")

print("\nSINR lengths statistics:")
print(f"Median: {sinr_median}")
print(f"Mean: {sinr_mean}")

RSSI lengths statistics:
Median: 36865.75
Mean: 34472.51296296297

SINR lengths statistics:
Median: 12727.0
Mean: 13178.524074074074


In [59]:
WINDOW_SIZE = 14000

In [60]:
def pad_or_truncate(data, target_length=WINDOW_SIZE):
    """
    Force time series to (target_length, 2) by:
      - Truncating if longer than target_length.
      - Padding with zeros if shorter.
    
    Args:
        data: Input array of shape (n_timesteps, 2).
        target_length: Desired length
    
    Returns:
        np.array: Padded/truncated data of shape (target_length, 2).
    """
    n_timesteps = data.shape[0]
    
    if n_timesteps > target_length:
        return data[:target_length]  # Truncate
    elif n_timesteps < target_length:
        return pad_sequences([data], maxlen=target_length, padding='post', dtype='float32')[0]  # Pad
    else:
        return data  # Already correct length

In [15]:
# Convert to fixed-size series of shape (14000, 2)
rssi_fixed = pad_or_truncate(rssi_reduced)
sinr_fixed = pad_or_truncate(sinr_reduced)
print(rssi_fixed.shape)
print(sinr_fixed.shape)

(14000, 2)
(14000, 2)


In [16]:
type(rssi_fixed)

numpy.ndarray

# Training

In [39]:
# Parameters
BATCH_SIZE = 8
EPOCHS = 100
VALIDATION_SPLIT = 0.2
RANDOM_SEED = 42

In [61]:
def calculate_group_size(rssi_length, sinr_length):
    """Calculate group size based on the ratio of lengths"""
    ratio = rssi_length / sinr_length
    # Round to nearest integer and ensure it's at least 1
    return max(1, int(np.round(ratio)))

In [69]:
# 1. Load and preprocess data
def load_data(base_path, num_samples=540):
    rssi_data = []
    sinr_data = []
    labels = one_hot_labels
    
    
    for i in range(1, num_samples + 1):
        # load data
        rssi_df = pd.read_csv(f"{base_path}/rssi_{i}.csv", header=0)
        sinr_df = pd.read_csv(f"{base_path}/sinr_{i}.csv", header=0)

        # Calculate dynamic group size based on length ratio
        group_size = calculate_group_size(len(rssi_df), len(sinr_df))
        print("Group size = ", group_size)

        # preprocess data
        rssi_df, sinr_df = reduce_and_match_shapes(rssi_df, sinr_df, group_size=group_size) # eg. (15946, 2)
        rssi_df = pad_or_truncate(rssi_df) # it becomes an ndarray after pad_truncate fn
        sinr_df = pad_or_truncate(sinr_df) # (14000, 2) 

        rssi_data.append(rssi_df)
        sinr_data.append(sinr_df)
    
    # Convert to numpy arrays
    rssi_data = np.array(rssi_data)  # array of arrays of shape (540, 14000, 2)
    sinr_data = np.array(sinr_data)
    labels = np.array(labels)
    
    return rssi_data, sinr_data, labels

In [63]:
# 2. Prepare the data
def prepare_data(rssi_data, sinr_data, labels):
    # Split into training and validation sets
    X_rssi_train, X_rssi_val, X_sinr_train, X_sinr_val, y_train, y_val = train_test_split(
        rssi_data, sinr_data, labels, 
        test_size=VALIDATION_SPLIT, 
        random_state=RANDOM_SEED,
        stratify=labels
    )
    
    return (X_rssi_train, X_sinr_train, y_train), (X_rssi_val, X_sinr_val, y_val)

In [65]:
# parameters
learning_rate = 0.025
dropout_rate = 0.4

# Convolutional block
def conv_block(inputs, filters, kernel_size, stride, regularization):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, padding="same", activation=None, kernel_regularizer=l2(regularization))(inputs)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

# Attention Block
def attention_block(inputs, n_heads, key_dim, regularization):
    x = MultiHeadAttention(num_heads=n_heads, key_dim=key_dim, kernel_regularizer=l2(regularization))(inputs, inputs)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    #x = GlobalAveragePooling1D()(x)
    return x

# Dense Block
def dense_block(inputs, units, noise_stddev, regularization):
    x = Dense(units, kernel_regularizer=l2(regularization))(inputs)
    x = GaussianNoise(stddev=noise_stddev)(x)  # Add Gaussian Noise
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x


# defines a HEAD of the mhdnn
def head(input):
    f = conv_block(input, 8, 6, 4, 1e-6)  #ip:(None, 14000, 2), op:(None, 7000, 8)
    f = conv_block(f, 16, 6, 4, 1e-6)     #ip:(None, 7000, 8), op:(None, 7000, 16)
    f = conv_block(f, 16, 5, 4, 1e-6)     #ip:(None, 7000, 16), op:(None, 3500, 16)
    a = attention_block(f, 4, 8, 1e-5)    #ip:(None, 3500, 16), op:(None, 3500, 16)---removed global avg pooling
    return a

# defines BODY of the mhdnn
def body(input):
    f = conv_block(input, 8, 3, 1, 1e-6)  #ip:(None, 3500, 32), op:(None, 3500, 8)
    f = conv_block(f, 16, 2, 1, 1e-6)     #ip:(None, 3500, 8), op:(None, 3500, 16)
    f = conv_block(f, 16, 2, 1, 1e-6)     #ip:(None, 3500, 16), op:(None, 3500, 16)
    f = Flatten()(f)                      #ip:(None, 3500, 16), op:(None, 56000)
    f = dense_block(f, 100, 0.3, 1e-5)    #ip:(None, 3500, 16), op:(None, 100)
    f = dense_block(f, 50, 0, 1e-5) # no gaussian noise here         #ip:(None, 100), op:(None, 50)
    # Output Layer with Softmax
    output = Dense(2, activation="softmax", name="output")(f)         #ip:(None, 50), op:(None, 2)
    return output

# Multi-Head Deep Neural Network (MH-DNN) Model
def build_mhdnn():
    input_shape = (WINDOW_SIZE, 2)
    input_rssi = Input(shape=input_shape, name="rssi_input") 
    input_sinr = Input(shape=input_shape, name="sinr_input")

    # pass through respective head
    rssi_attn = head(input_rssi)
    sinr_attn = head(input_sinr)

    # Ensure shapes are correct before concatenation
    print("RSSI head output shape:", rssi_attn.shape)  # Debug
    print("SINR head output shape:", sinr_attn.shape)  # Debug

    # Concatenate extracted features
    merged_features = Concatenate(axis=-1)([rssi_attn, sinr_attn])  #op:(None, 3500, 32)

    print("Merged features shape:", merged_features.shape)  # Debug

    # pass through body of mhdnn
    output = body(merged_features)

    # Build model
    model = Model(inputs=[input_rssi, input_sinr], outputs=output, name="MH-DNN")
    return model

In [66]:
# 3. Build and compile the model
def get_compiled_model():
    model = build_mhdnn()
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [67]:
# 4. Training callbacks
def get_callbacks():
    checkpoint = ModelCheckpoint(
        'best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
    
    return [checkpoint, early_stopping, reduce_lr]

In [70]:
# Load data
base_path = "dataset/training"
rssi_data, sinr_data, labels = load_data(base_path)

Group size =  4
RSSI reduced shape: (13965, 2) (from original (63862, 2))
Truncated SINR to match RSSI: (13965, 2)
Group size =  4
RSSI reduced shape: (13965, 2) (from original (63862, 2))
Truncated SINR to match RSSI: (13965, 2)
Group size =  4
RSSI reduced shape: (13965, 2) (from original (63862, 2))
Truncated SINR to match RSSI: (13965, 2)
Group size =  6
RSSI reduced shape: (11968, 2) (from original (87808, 2))
Truncated SINR to match RSSI: (11968, 2)
Group size =  6
RSSI reduced shape: (11968, 2) (from original (87808, 2))
Truncated SINR to match RSSI: (11968, 2)
Group size =  6
RSSI reduced shape: (11968, 2) (from original (87808, 2))
Truncated SINR to match RSSI: (11968, 2)
Group size =  6
RSSI reduced shape: (11968, 2) (from original (87808, 2))
Truncated SINR to match RSSI: (11968, 2)
Group size =  8
RSSI reduced shape: (11841, 2) (from original (118731, 2))
Truncated SINR to match RSSI: (11841, 2)
Group size =  7
RSSI reduced shape: (13528, 2) (from original (118700, 2))
Trun

In [71]:
print("RSSI data shape:", rssi_data.shape)  # Should be (540, 14000, 2)
print("SINR data shape:", sinr_data.shape)  # Should be (540, 14000, 2)
print("Labels shape:", labels.shape)  # Should be (540,2)

RSSI data shape: (540, 14000, 2)
SINR data shape: (540, 14000, 2)
Labels shape: (540, 2)


In [72]:
# Prepare data
(X_rssi_train, X_sinr_train, y_train), (X_rssi_val, X_sinr_val, y_val) = prepare_data(rssi_data, sinr_data, labels)

In [73]:
# Build model
model = get_compiled_model()
model.summary()

RSSI head output shape: (None, 219, 16)
SINR head output shape: (None, 219, 16)
Merged features shape: (None, 219, 32)
Model: "MH-DNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rssi_input (InputLayer)        [(None, 14000, 2)]   0           []                               
                                                                                                  
 sinr_input (InputLayer)        [(None, 14000, 2)]   0           []                               
                                                                                                  
 conv1d_27 (Conv1D)             (None, 3500, 8)      104         ['rssi_input[0][0]']             
                                                                                                  
 conv1d_30 (Conv1D)             (None, 3500, 8)      104         ['sinr_i

In [74]:
# Train model
history = model.fit(
    [X_rssi_train, X_sinr_train],
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=([X_rssi_val, X_sinr_val], y_val),
    callbacks=get_callbacks(),
    verbose=1
)

Epoch 1/100
53/54 [============================>.] - ETA: 0s - loss: 0.8165 - accuracy: 0.5094
Epoch 1: val_accuracy improved from -inf to 0.60185, saving model to best_model.h5
54/54 [==============================] - 5s 51ms/step - loss: 0.8196 - accuracy: 0.5069 - val_loss: 0.6836 - val_accuracy: 0.6019 - lr: 0.0010
Epoch 2/100
54/54 [==============================] - ETA: 0s - loss: 0.7232 - accuracy: 0.5671
Epoch 2: val_accuracy did not improve from 0.60185
54/54 [==============================] - 2s 40ms/step - loss: 0.7232 - accuracy: 0.5671 - val_loss: 0.6772 - val_accuracy: 0.6019 - lr: 0.0010
Epoch 3/100
53/54 [============================>.] - ETA: 0s - loss: 0.7241 - accuracy: 0.5755
Epoch 3: val_accuracy did not improve from 0.60185
54/54 [==============================] - 2s 41ms/step - loss: 0.7226 - accuracy: 0.5741 - val_loss: 0.6740 - val_accuracy: 0.6019 - lr: 0.0010
Epoch 4/100
53/54 [============================>.] - ETA: 0s - loss: 0.6785 - accuracy: 0.6108
Epoch 

In [75]:
# Save the final model
model.save('mhdnn_final_model.h5')
print("Training completed and model saved.")

Training completed and model saved.


# Testing
We obtained the best accuracy of 78.24% during training. Now we use this to predict output on certain testcases.

In [76]:
def load_test_data(rssi_path, sinr_path):
    """Load and preprocess single test sample"""
    # Load CSV files
    rssi_df = pd.read_csv(rssi_path, header=0)
    sinr_df = pd.read_csv(sinr_path, header=0)
    
    # Preprocess to match training format
    rssi_df, sinr_df = reduce_and_match_shapes(rssi_df, sinr_df, group_size=4)
    rssi_data = pad_or_truncate(rssi_df)  # Should return (14000, 2)
    sinr_data = pad_or_truncate(sinr_df)  # Should return (14000, 2)
    
    # Add batch dimension
    rssi_data = np.expand_dims(rssi_data, axis=0)  # (1, 14000, 2)
    sinr_data = np.expand_dims(sinr_data, axis=0)  # (1, 14000, 2)
    
    return rssi_data, sinr_data

In [77]:
def predict_attack(rssi_path, sinr_path, model_path='mhdnn_final_model.h5'):
    """
    Make prediction on new RSSI/SINR data
    Args:
        rssi_path: path to RSSI CSV file
        sinr_path: path to SINR CSV file
        model_path: path to saved Keras model
    Returns:
        prediction: 0 or 1
        confidence: probability score [0-1]
    """
    # Load model
    model = load_model(model_path, compile=False)
    
    # Load and preprocess test data
    rssi_data, sinr_data = load_test_data(rssi_path, sinr_path)
    
    # Make prediction (now returns [prob_class0, prob_class1])
    predictions = model.predict([rssi_data, sinr_data])
    predicted_class = np.argmax(predictions[0])  # Returns 0 or 1
    confidence = predictions[0][predicted_class]  # Get the probability
    
    return predicted_class, confidence

In [84]:
def pred_interpret(pred, conf, uav_id="UAV-3"):
    if pred == 1 and conf > 0.7:
        return f"{uav_id}: Jamming attack occurred!"
    else:
        return f"{uav_id}: No attack occurred."

In [85]:
# Paths to your test files
test_rssi_path1 = "dataset/attack/TC4/UE1_rssi.csv"
test_sinr_path1 = "dataset/attack/TC4/UE1_sinr.csv"
test_rssi_path2 = "dataset/attack/TC4/UE2_rssi.csv"
test_sinr_path2 = "dataset/attack/TC4/UE2_sinr.csv"
test_rssi_path3 = "dataset/attack/TC4/UE3_rssi.csv"
test_sinr_path3 = "dataset/attack/TC4/UE3_sinr.csv"
test_rssi_path4 = "dataset/attack/TC4/UE4_rssi.csv"
test_sinr_path4 = "dataset/attack/TC4/UE4_sinr.csv"
test_rssi_path5 = "dataset/attack/TC4/UE5_rssi.csv"
test_sinr_path5 = "dataset/attack/TC4/UE5_sinr.csv"
test_rssi_path6 = "dataset/attack/TC4/UE6_rssi.csv"
test_sinr_path6 = "dataset/attack/TC4/UE6_sinr.csv"

# Make predictions for all 6 UAVs
pred, conf = predict_attack(test_rssi_path1, test_sinr_path1)
print(f"Prediction: {pred} (Confidence: {conf:.4f})")
print(pred_interpret(pred, conf, "UAV-1"))
print()

pred, conf = predict_attack(test_rssi_path2, test_sinr_path2)
print(f"Prediction: {pred} (Confidence: {conf:.4f})")
print(pred_interpret(pred, conf, "UAV-2"))
print()

pred, conf = predict_attack(test_rssi_path3, test_sinr_path3)
print(f"Prediction: {pred} (Confidence: {conf:.4f})")
print(pred_interpret(pred, conf, "UAV-3"))
print()

pred, conf = predict_attack(test_rssi_path4, test_sinr_path4)
print(f"Prediction: {pred} (Confidence: {conf:.4f})")
print(pred_interpret(pred, conf, "UAV-4"))
print()

pred, conf = predict_attack(test_rssi_path5, test_sinr_path5)
print(f"Prediction: {pred} (Confidence: {conf:.4f})")
print(pred_interpret(pred, conf, "UAV-5"))
print()

pred, conf = predict_attack(test_rssi_path6, test_sinr_path6)
print(f"Prediction: {pred} (Confidence: {conf:.4f})")
print(pred_interpret(pred, conf, "UAV-6"))
print()

RSSI reduced shape: (26250, 2) (from original (137003, 2))
Truncated RSSI to match SINR: (14363, 2)
Prediction: 1 (Confidence: 0.9999)
UAV-1: Jamming attack occurred!

RSSI reduced shape: (26212, 2) (from original (136848, 2))
Truncated RSSI to match SINR: (14518, 2)
Prediction: 1 (Confidence: 1.0000)
UAV-2: Jamming attack occurred!

RSSI reduced shape: (26128, 2) (from original (136514, 2))
Truncated RSSI to match SINR: (14853, 2)
Prediction: 1 (Confidence: 0.6064)
UAV-3: No attack occurred.

RSSI reduced shape: (25975, 2) (from original (135902, 2))
Truncated RSSI to match SINR: (16035, 2)
Prediction: 1 (Confidence: 1.0000)
UAV-4: Jamming attack occurred!

RSSI reduced shape: (26135, 2) (from original (136542, 2))
Truncated RSSI to match SINR: (14823, 2)
Prediction: 1 (Confidence: 0.9093)
UAV-5: Jamming attack occurred!

RSSI reduced shape: (25097, 2) (from original (132391, 2))
Truncated RSSI to match SINR: (21080, 2)
Prediction: 0 (Confidence: 0.5307)
UAV-6: No attack occurred.

